In [1]:
import pandas as pd
import fitz
import os
import pickle
import math
from bs4 import BeautifulSoup
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier

In [2]:
%cd "c:\\Users\\t1nipun\\Documents\\esa-data-bank_banque-donnees-ees"

c:\Users\t1nipun\Documents\esa-data-bank_banque-donnees-ees


In [3]:
#### Reading the Index 1 file - Dataframe
path = os.getcwd()

#features = pd.read_csv(path + '\\data\\processed\\page_features\\all_features.csv', index_col = 0, encoding= 'unicode_escape')
features = pd.read_csv(path + '\\data\\processed\\page_features\\all_features.csv', index_col = 0)
print(len(features))
features.head()

2264


,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,dataID,pageNo,paths_l
0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,0,0,4166536,1,c:\Users\t1nipun\Documents\esa-data-bank_banqu...
1,0,0,0,0,0,0,0,0,0,0,0,1,0,541,1,12980,4166536,2,c:\Users\t1nipun\Documents\esa-data-bank_banqu...
2,0,0,0,0,0,0,0,0,0,0,0,0,0,375,1,4160,4166536,3,c:\Users\t1nipun\Documents\esa-data-bank_banqu...
3,0,0,0,2,1,0,0,0,0,0,0,8,0,1060,1,4160,4166536,4,c:\Users\t1nipun\Documents\esa-data-bank_banqu...
4,1,0,0,0,1,0,0,0,0,0,1,0,0,956,1,4160,4166536,5,c:\Users\t1nipun\Documents\esa-data-bank_banqu...


In [4]:
X_df_features = features.copy()
#X_df_features.drop(columns=['dataID_pageNo', 'result'], inplace=True)
X_df_features.drop(columns=['dataID', 'pageNo', 'paths_l'], inplace=True)
X_df_features.head()

,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images
0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,541,1,12980
2,0,0,0,0,0,0,0,0,0,0,0,0,0,375,1,4160
3,0,0,0,2,1,0,0,0,0,0,0,8,0,1060,1,4160
4,1,0,0,0,1,0,0,0,0,0,1,0,0,956,1,4160


In [5]:
m_path = path + '\\models\\alignment_sheet_classifier_rfc.sav'
loaded_model = pickle.load(open(m_path, 'rb'))
result = loaded_model.predict(X_df_features)

y_predict = []
for y in result:
    if y > 0.50:
        y_predict.append(1)
    else:
        y_predict.append(0)
        
print(len(y_predict))
print(y_predict[:5])
print(sum(y_predict))

features["results"] = y_predict

2264
[0, 0, 0, 0, 0]
327


c:\Users\t1nipun\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\Users\t1nipun\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
alignment_sheets = features[features.results > 0.5]
print("Total number of Pages: ", len(features))
print("Total number of Alignment Sheets: ", len(alignment_sheets))
alignment_sheets.reset_index(drop=True, inplace=True)
alignment_sheets.head()

Total number of Pages:  2264
Total number of Alignment Sheets:  327


,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,dataID,pageNo,paths_l,results
0,0,1,0,1,1,0,2,0,0,0,1,6,0,4667,4,1154441,4166218,5,c:\Users\t1nipun\Documents\esa-data-bank_banqu...,1
1,0,1,0,1,1,0,2,0,0,0,1,6,0,7358,4,1154441,4166218,6,c:\Users\t1nipun\Documents\esa-data-bank_banqu...,1
2,0,1,0,1,1,0,1,0,0,0,1,3,0,5836,5,2141830,4166301,231,c:\Users\t1nipun\Documents\esa-data-bank_banqu...,1
3,0,2,0,3,1,1,0,0,0,2,1,3,0,18711,13,1462433,4166834,5,c:\Users\t1nipun\Documents\esa-data-bank_banqu...,1
4,0,2,0,3,1,1,0,0,0,2,1,3,0,14705,13,1462433,4166834,8,c:\Users\t1nipun\Documents\esa-data-bank_banqu...,1


In [7]:
alignment_sheets_list = []
for index, row in alignment_sheets.iterrows():
    al = str(row["dataID"]) + "_" + str(row["pageNo"])
    alignment_sheets_list.append(al)
    
print(len(alignment_sheets_list))

327


In [25]:
value = '4166301'
result = [i for i in alignment_sheets_list if i.startswith(value)]
result

['4166301_231']

### Check if alignment sheets were previously extracted as Figure

In [37]:
#index_file = pd.read_csv(path + "Data_Files_v2\\book1.csv", index_col = 0, encoding= 'unicode_escape')
index_file = pd.read_csv(path + '\\data\\interim\\Intermediate_Index_Files\\table_figs_index.csv', encoding = 'utf-8-sig')
index_file[["PDF Page Number", "tableNumber", "figureNumber"]] = index_file[["PDF Page Number", "tableNumber", "figureNumber"]].astype('Int64')
print(len(index_file))
index_file.head()

706


,Title,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,...,Decision URL,ESA Section(s),Application ID,Topics,PDF Page Number,tableNumber,csvFileName,PDF Page Count,figureNumber,Content Type
0,Table 7.28 Residual Cumulative Effects on Wild...,Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F7,4166218,...,NaN,APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ...,59480,"['Wildlife', 'Human']",10,1,4166218_10_lattice-v_1.csv,20,NaN,Table
1,Table 7.27 Change in Linear Features Density w...,Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F7,4166218,...,NaN,APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ...,59480,"['Wildlife', 'Human']",4,1,4166218_4_lattice-v_1.csv,20,NaN,Table
2,"Table 4.2 in Section 4.7.1, Assessment Methods...",Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F6,4166300,...,NaN,APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ...,59480,"['Wildlife', 'Human']",2,1,4166300_2_lattice-v_1.csv,10,NaN,Table
3,Table 7.26 Change in Land Cover Classes within...,Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F6,4166300,...,NaN,APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ...,59480,"['Wildlife', 'Human']",6,1,4166300_6_lattice-v_1.csv,10,NaN,Table
4,Table 8.2 Characterization of Residual Effects...,Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-7,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F8,4166301,...,NaN,APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ...,59480,"['Land', 'Air', 'Water', 'Wildlife', 'Human', ...",10,1,4166301_10_lattice-v_1.csv,531,NaN,Table


In [39]:
for index, row in index_file.iterrows():
    dataID_value = row["PDF Download URL"].split("/")[-1]
    pageno_value = row["PDF Page Number"]
    sheet = str(dataID_value) + "_" + str(pageno_value)
    if sheet in alignment_sheets_list:
        alignment_sheets_list.remove(sheet)
        
        index_file.loc[index, 'Content Type'] = "Alignment Sheet"

In [40]:
len(alignment_sheets_list)

326

In [41]:
as_count = 0
for index, row in index_file.iterrows():
    if "Alignment Sheet"  in row["Content Type"]:
        as_count = as_count+1
 
print(as_count)

1


### For the rest of the alignment sheets - Extract the titles

In [43]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def count_digits_in_string(word):
    digit_count = 0
    for a in word:
        if a.isdigit():
            digit_count = digit_count + 1
    return(digit_count)

def get_alignment_sheet_title(dataID, pageNo, path):
    path_pkl = path + '\\data\\processed\\pickle_files\\' + str(dataID) + ".pkl"
    title_found = 0
    table_title = ""
    table_title_next = ""
    ytest_no_blanks = ""
    last_ptag_starts_with_key = 0    
    with open(path_pkl, 'rb') as f:
        data = pickle.load(f)
        soup = BeautifulSoup(data['content'], 'lxml')
        pages = soup.find_all('div', attrs={'class': 'page'})
        for b, p in enumerate(pages):
            if b != pageNo - 1:
                continue
                
            #print(dataID, pageNo)
                
            pages_text = [x.text for x in p.find_all('p')]
            for y in pages_text:
                    
                pages_text = [x.text for x in p.find_all('p')]
                for y in pages_text:
                    ytest = y.replace('\n','').replace('\r','')
                    ytest = ytest.split(" ")
                    ytest_no_blanks = ""
                    for ytes in ytest:
                        if len(ytes) > 2:
                            ytest_no_blanks = ytest_no_blanks + ytes + " "
                            
                    if last_ptag_starts_with_key ==1:
                        table_title_next = (y.replace('\n',' ').replace('\r',''))
                        break
                        
                    if ytest_no_blanks.startswith(('FIGURE', 'Figure', 'PHOTO', 'Photo')):
                        #and not any(substring in y for substring in exceptions_list):
                        table_title = y.replace('\n','').replace('\r','')
                        last_ptag_starts_with_key = 1
                        title_found = 1
                        
        alternate_title = "Alignment Sheet " + str(dataID) + " " + str(pageNo)
        return(table_title, table_title_next, alternate_title)
        

In [ ]:
# def decide_title(table_title, table_title_next, alternate_title):
     
#     table_string = table_title.replace('\n','').replace('\r','').replace('–',"").replace(":","").replace("-","").replace("(","")
#     table_string = table_string.replace('.','')
#     category = 0
        
#     title_words = re.split(" ", table_string)
        
#     title_words_no_blanks = []
#     for title_word in title_words:
#         if len(title_word) > 2:
#             title_words_no_blanks.append(title_word)
#         elif len(title_word) > 0 and count_digits_in_string(title_word) > 0:
#                 title_words_no_blanks.append(title_word)
                
    
        
#     category = 0
#     if len(title_words_no_blanks) < 3: 
#             category = 0.75
            
#     if 'cont' in table_string.lower():
#         category = 0.35
            
        
#     #category2 = 0
#     if len(title_words_no_blanks) > 2:
#         if(title_words_no_blanks[2][0].isupper() or title_words_no_blanks[2][0].isdigit()):
#             category = 1
#         else:
#             category = 0
#         if(title_words_no_blanks[1][-1]== ',') or (title_words_no_blanks[1][-1]== ']') or (title_words_no_blanks[1][-1]== ')'):
#             category = 0
#         if(title_words_no_blanks[2][:4].lower() == 'cont'):
#                 category = 0.35
            
#     if len(title_words_no_blanks) == 3 :
#         category = 0.5
#         if(title_words_no_blanks[1][0].isdigit() and  title_words_no_blanks[1][0].isupper()):
#             category = 0.65
#         if(title_words_no_blanks[2][0].lower() == 'c'):
#             category = 0.55
#         if(title_words_no_blanks[2][:4].lower() == 'cont'):
#             category = 0.35
            
#     if category < 0.5 or len(title_words_no_blanks) < 6 or count_digits_in_string(title_words_no_blanks) > 10:
#         return(alternate_title, 0)
#     elif category < 0.7:
#         return(table_title + table_title_next, 1)
#     else:
#         return(table_title, 1)
        
        

In [44]:
def decide_title(table_title, table_title_next, alternate_title):
    
    if count_digits_in_string(table_title) > 10:
        return(alternate_title, 0)
     
    table_string = table_title.replace('\n','').replace('\r','').replace('–',"").replace(":","").replace("-","").replace("(","")
    table_string = table_string.replace('.','')
    category = 0
        
    title_words = table_string.split()
        
    title_words_no_blanks = []
    for title_word in title_words:
        if len(title_word) > 2:
            title_words_no_blanks.append(title_word)
                
    category = 0
    if len(title_words_no_blanks) < 3: 
        category = 0.45
            
    if 'cont' in table_string.lower():
        category = 0.35
            
        
    #category2 = 0
    if len(title_words_no_blanks) > 2:
        if(title_words_no_blanks[2][0].isupper() or title_words_no_blanks[2][0].isdigit()):
            category = 1
        else:
            category = 0
        if(title_words_no_blanks[1][-1]== ',') or (title_words_no_blanks[1][-1]== ']') or (title_words_no_blanks[1][-1]== ')'):
            category = 0
        if(title_words_no_blanks[2][:4].lower() == 'cont'):
            category = 0.35
            
    if len(title_words_no_blanks) == 3 :
        category = 0.5
        if(title_words_no_blanks[1][0].isdigit() and  title_words_no_blanks[1][0].isupper()):
            category = 0.65
        if(title_words_no_blanks[2][0].lower() == 'c'):
            category = 0.55
        if(title_words_no_blanks[2][:4].lower() == 'cont'):
            category = 0.35
            
    if category < 0.5 or len(title_words_no_blanks) < 6 or count_digits_in_string(title_words_no_blanks) > 10:
        return(alternate_title, 0)
    elif category < 0.7:
        return(table_title + table_title_next, 1)
    else:
        return(table_title, 1)
        
        

In [ ]:
"Ref-> https://github.com/CER-REC/esa-data-bank_banque-donnees-ees/blob/3a59e33b1ee9a37857380387d7c380e9139eedbf/Codes/TableCategorization.py"

In [ ]:
# for alignment_s in alignment_sheets_list:
#     dataid_alignment_s, pageno_alignment_s = alignment_s.split("_")
#     table_title, table_title_next, alternate_title = get_alignment_sheet_title(int(dataid_alignment_s), int(pageno_alignment_s), path)
#     a, b = decide_title(table_title, table_title_next, alternate_title)
#     print(dataid_alignment_s, pageno_alignment_s,"------Final Title: ",b, a)

In [45]:
Title = []
Content_Type = []
PDF_Page_Number = []
Data_ID = []
#thumbnail_location = []
needs_translation = []


#alignment_sheets_list = alignment_sheets_list[:50]
count = 0
for alignment_s in alignment_sheets_list:
    #print("_____________", count, " of ", len(alignment_sheets_list) )
    count = count +1
    dataid_alignment_s, pageno_alignment_s = alignment_s.split("_")
    
    #Extrct the title of the alignment sheet
    table_title, table_title_next, alternate_title = get_alignment_sheet_title(int(dataid_alignment_s), int(pageno_alignment_s), path)
    titl, needs_transla = decide_title(table_title, table_title_next, alternate_title)
    #print(needs_transla, titl)
    Title.append(titl)
    needs_translation.append(needs_transla)
    Content_Type.append(str("Alignment Sheet"))
    PDF_Page_Number.append(int(pageno_alignment_s))
    Data_ID.append(int(dataid_alignment_s))
    #thumbnail_location.append("thumbnails\\"+ alignment_s + ".jpg")

In [46]:
# for index, row in alignment_sheets.iterrows():
#     Id.append(str(row["dataID"]) + "_" + str(row["pageNo"]) + "_A")
#     #print("alignment_sheets_thumbnails\\"+ str(row["dataID"]) + "_" +str(row["pageNo"])+ ".jpg")
    
as_index_for_translation = pd.DataFrame({'Title' : Title, 
                                         'ID' : alignment_sheets_list,
                                         'Needs_Translation': needs_translation})
print(len(as_index_for_translation))
as_index_for_translation = as_index_for_translation[as_index_for_translation.Needs_Translation > 0.5]
as_index_for_translation.drop('Needs_Translation', 
                               axis=1, 
                               inplace=True)

as_index_for_translation.to_csv(path + "\\data\\interim\\alignment_sheet_files\\new_alignment_sheet_titles_for_translation.csv",
                               index=False)

326


In [47]:
print(len(as_index_for_translation))

0


### For the rest of the alignment sheets - Extract Other Index Coloumn Values from PDF Table

In [48]:
index_pdf_df = pd.read_csv(path + '\\data\\interim\\Intermediate_Index_Files\\Index_of_PDFs_for_Fall_2022_application_refresh_processed.csv')
index_pdf_df = index_pdf_df.rename(columns={"Data ID": "DataID"})
index_pdf_df.head()

,application_name,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,DataID,...,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,Application ID,Topics,PDF Size (bytes),Number of Pages,Outline Present
0,2021-11-18 - Application for the NEBC Connecto...,Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-4,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F5,4166536,...,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,NaN,APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ...,NaN,59480,"['Land', 'Water', 'Wildlife', 'Vegetation', 'H...",5823639,293,1
1,2021-11-18 - Application for the NEBC Connecto...,Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F6,4166300,...,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,NaN,APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ...,NaN,59480,"['Wildlife', 'Human']",17824235,10,0
2,2021-11-18 - Application for the NEBC Connecto...,Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F7,4166218,...,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,NaN,APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ...,NaN,59480,"['Wildlife', 'Human']",3197740,20,1
3,2021-11-18 - Application for the NEBC Connecto...,Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-7,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F8,4166301,...,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,NaN,APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ...,NaN,59480,"['Land', 'Air', 'Water', 'Wildlife', 'Human', ...",10570168,531,1
4,2021-11-18 - Application for the NEBC Connecto...,Application for the NEBC Connector Project,NEBC Connector,11/18/2021,NorthRiver Midstream NEBC Connector GP Inc.,Gas,C16186-8,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,A7Y7F9,4166833,...,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,NaN,APPENDIX A ENVIRONMENTAL PROTECTION PLAN,NaN,59480,['Environment Protection Plan'],5975398,160,1


In [65]:
Application_Name = []
Application_Short_Name = []
Application_Filing_Date = []
Company_Name = []
Commodity = []
File_Name = []
ESA_Folder_URL =[]
Document_Number = []
PDF_Download_URL =[]
Application_Type = []
Pipeline_Location =[]
Hearing_order =[]
Consultant_Name =[]
Pipeline_Status =[]
Regulatory_Instrument =[]
Application_URL =[]
Decision_URL =[]
ESA_Section = []
Application_ID = []
Topics = []
PDF_Page_Count =[]
# Landscape_terrain_and_weather = []
# Soil = []
# Plants = []
# Water =[]
# Fish = []
# Wetlands = []
# Wildlife = []
# Species_at_Risk = []
# Greenhouse_gas_emissions = []
# Air_emissions = []
# Noise = []
# Electricity_and_electromagnetism = []
# Proximity_to_people = []
# Archaeological_aleontological = []
# Human_access  = []
# Indigenous_land_water_and_air_use = []
# Impact_to_social_and_cultural_well_being = []
# Impact_to_human_health_and_viewscapes = []
# Social_cultural_economic_infrastructure_and_services = []
# Economic_Offsets_and_Impact = []
# Environmental_Obligations = []
# Treaty_and_Indigenous_Rights = []
# Project_Download_Path = []
# Table_Download_Path = []
# Good_Quality = []

In [66]:
for alignment_s in alignment_sheets_list:
    dataid_alignment_s, pageno_alignment_s = alignment_s.split("_")
    
    pdf_df = index_pdf_df[index_pdf_df.DataID == int(dataid_alignment_s)]
    if len(pdf_df) ==0:
        print("DataID not found in pdf index: ", dataid_alignment_s)
        continue
    if len(pdf_df) > 1:
        print("Duplicate DataIDs found in pdf index: ", dataid_alignment_s)
        continue
        
    
    Application_Name.append(pdf_df["Application Name"].iloc[0])
    Application_Short_Name.append(pdf_df["Application Short Name"].iloc[0])
    Application_Filing_Date.append(pdf_df["Application Filing Date"].iloc[0])
    Company_Name.append(pdf_df["Company Name"].iloc[0])
    Commodity.append(pdf_df["Commodity"].iloc[0])
    File_Name.append(pdf_df["File Name"].iloc[0])
    ESA_Folder_URL.append(pdf_df["ESA Folder URL"].iloc[0])
    Document_Number.append(pdf_df["Document Number"].iloc[0])
    PDF_Download_URL.append(pdf_df["PDF Download URL"].iloc[0])
    Application_Type.append(pdf_df["Application Type"].iloc[0])
    Pipeline_Location.append(pdf_df["Pipeline Location"].iloc[0])
    Hearing_order.append(pdf_df["Hearing Order"].iloc[0])
    Consultant_Name.append(pdf_df["Consultant Name"].iloc[0])
    Pipeline_Status.append(pdf_df["Pipeline Status"].iloc[0])
    Regulatory_Instrument.append(pdf_df["Regulatory Instrument(s)"].iloc[0])
    Application_URL.append(pdf_df["Application URL"].iloc[0])
    Decision_URL.append(pdf_df["Decision URL"].iloc[0])
    ESA_Section.append(pdf_df["ESA Section(s)"].iloc[0])
    Application_ID.append(pdf_df["Application ID"].iloc[0])
    Topics.append(pdf_df["Topics"].iloc[0])
    PDF_Page_Count.append(pdf_df["Number of Pages"].iloc[0])
    # Landscape_terrain_and_weather.append(np.nan)
    # Soil.append(np.nan)
    # Plants.append(np.nan)
    # Water.append(np.nan)
    # Fish.append(np.nan)
    # Wetlands.append(np.nan)
    # Wildlife.append(np.nan)
    # Species_at_Risk.append(np.nan)
    # Greenhouse_gas_emissions.append(np.nan)
    # Air_emissions.append(np.nan)
    # Noise.append(np.nan)
    # Electricity_and_electromagnetism.append(np.nan)
    # Proximity_to_people.append(np.nan)
    # Archaeological_aleontological.append(np.nan)
    # Human_access.append(np.nan)
    # Indigenous_land_water_and_air_use.append(np.nan)
    # Impact_to_social_and_cultural_well_being.append(np.nan)
    # Impact_to_human_health_and_viewscapes.append(np.nan)
    # Social_cultural_economic_infrastructure_and_services.append(np.nan)
    # Economic_Offsets_and_Impact.append(np.nan)
    # Environmental_Obligations.append(np.nan)
    # Treaty_and_Indigenous_Rights.append(np.nan)
    
    # Project_Download_Path.append(np.nan)
    # Table_Download_Path.append(np.nan)
    # Good_Quality.append(np.nan)
    
    

In [67]:
print(len(Title))
print(len(Content_Type))
print(len(Application_Name))
#print(len(Project_Download_Path))

326
326
326


In [68]:
index_as_df = pd.DataFrame({'Title' : Title, 
                            'Content Type' : Content_Type, 
                            'Application Name' : Application_Name,
                            'Application Short Name' : Application_Short_Name, 
                            'Application Filing Date' :Application_Filing_Date, 
                            'Company Name' : Company_Name, 
                            'Commodity' :  Commodity, 
                            'File Name' : File_Name, 
                            'ESA Folder URL' : ESA_Folder_URL, 
                            'Document Number' : Document_Number, 
                            'Data ID' : Data_ID, 
                            'PDF Download URL' : PDF_Download_URL, 
                            'Application Type' : Application_Type, 
                            'Pipeline Location' : Pipeline_Location, 
                            'Hearing Order' : Hearing_order, 
                            'Consultant Name' : Consultant_Name, 
                            'Pipeline Status' : Pipeline_Status, 
                            'Regulatory Instrument(s)' : Regulatory_Instrument, 
                            'Application URL' : Application_URL,
                            'Decision URL': Decision_URL, 
                            'ESA Section(s)' : ESA_Section, 
                            'Application ID': Application_ID,
                            'Topics' : Topics, 
                            'PDF Page Count' : PDF_Page_Count, 
                            
                            
                            # 'Landscape, terrain, and weather' : Landscape_terrain_and_weather,
                            # 'Soil' : Soil,
                            # 'Plants' : Plants,
                            # 'Water' : Water, 
                            # 'Fish' : Fish,
                            # 'Wetlands' : Wetlands,
                            # 'Wildlife' : Wildlife, 
                            # 'Species at Risk' : Species_at_Risk, 
                            # 'Greenhouse gas emissions' : Greenhouse_gas_emissions,
                            # 'Air emissions' : Air_emissions,
                            # 'Noise' : Noise,
                            # 'Electricity and electromagnetism' : Electricity_and_electromagnetism,
                            # 'Proximity to people' : Proximity_to_people,
                            # 'Archaeological, paleontological, historical, and culturally significant sites and resources' : Archaeological_aleontological,
                            # 'Human access to boats and waterways' : Human_access,
                            # 'Indigenous land, water, and air use' : Indigenous_land_water_and_air_use,
                            # 'Impact to social and cultural well-being' : Impact_to_social_and_cultural_well_being,
                            # 'Impact to human health and viewscapes' : Impact_to_human_health_and_viewscapes,
                            # 'Social, cultural, economic infrastructure and services': Social_cultural_economic_infrastructure_and_services,
                            # 'Economic Offsets and Impact' : Economic_Offsets_and_Impact, 
                            # 'Environmental Obligations' : Environmental_Obligations,
                            # 'Treaty and Indigenous Rights' : Treaty_and_Indigenous_Rights,
                            
                            
                            # 'Project Download Path' : Project_Download_Path, 
                            # 'Table Download Path' : Table_Download_Path, 
                            # 'Good Quality': Good_Quality,
                            })

index_as_df.to_csv(path + "\\data\\interim\\alignment_sheet_files\\Index_Alignment_Sheets_try.csv", index = False)

### Merge with the index file of tables and figures

In [69]:
df_table_fig_align_index = index_file.append(index_as_df, ignore_index=True)

c:\Users\t1nipun\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [71]:
df_table_fig_align_index.columns

Index(['Application Filing Date', 'Application ID', 'Application Name',
       'Application Short Name', 'Application Type', 'Application URL',
       'Commodity', 'Company Name', 'Consultant Name', 'Content Type',
       'Data ID', 'Decision URL', 'Document Number', 'ESA Folder URL',
       'ESA Section(s)', 'File Name', 'Hearing Order', 'PDF Download URL',
       'PDF Page Count', 'PDF Page Number', 'Pipeline Location',
       'Pipeline Status', 'Regulatory Instrument(s)', 'Title', 'Topics',
       'csvFileName', 'figureNumber', 'tableNumber'],
      dtype='object')

In [78]:
#df_table_fig_align_index['Content Type'] = df_table_fig_align_index['Content Type'].replace('Figure', 'Alignment Sheet')
for idx, row in df_table_fig_align_index.iterrows():
    if row['Content Type'] == 'Alignment Sheet' and row['Title'].startswith('Fig'):
        df_table_fig_align_index.loc[idx, 'Title'] = '(Alignment Sheet) ' + row['Title']

### Generate ID and ID Internal Columns

In [204]:
#df_table_fig_align_index['alignNumber'] = df_table_fig_align_index['Content Type'].apply(lambda x: int(1) if x == 'Alignment Sheet' else '')
#df_table_fig_align_index.insert(len(df_table_fig_align_index.columns), 'ID', range(21642, 21642 + len(df_table_fig_align_index)))
# df_table_fig_align_index['ID Internal'] = ''
# for idx, row in df_table_fig_align_index.iterrows():
#     if row['Content Type'] == 'Table':
#         df_table_fig_align_index.loc[idx, 'ID Internal'] = str(row['Data ID']) + '_' + str(row['PDF Page Number']) + '_t_' + str(int(row.tableNumber))
#     elif row['Content Type'] == 'Alignment Sheet':
#         df_table_fig_align_index.loc[idx, 'ID Internal'] = str(row['Data ID']) + '_' + str(row['PDF Page Number']) + '_a_' + str(int(row.alignNumber))
#     else:
#         df_table_fig_align_index.loc[idx, 'ID Internal'] = str(row['Data ID']) + '_' + str(row['PDF Page Number']) + '_f_' + str(int(row.figureNumber))

#df_table_fig_align_index.to_csv(path + "\\data\\interim\\Intermediate_Index_Files\\Table_Fig_Alignment_Index.csv", index = False)

In [73]:
df_table_fig_align_index.to_csv(path + "\\data\\interim\\Intermediate_Index_Files\\Table_Fig_Alignment_Index.csv", index = False)

### Create thumbnails for figures/tables/alignment sheets

In [ ]:
def get_thumbnail(dataID, pageNo, path):
    
    file_p = path + "Data_Files_v2\\PDFs\\"+ str(dataID) + ".pdf"
    doc = fitz.open(file_p)
    for page_no in range(1, len(doc)+1):   
        if page_no != pageNo:
            continue
        
        page = doc.loadPage(page_no-1)  # number of page
        pix = page.getPixmap()
        
        #Code to save the Thumbnails = 250 * 176
        if pix.width < pix.height:
            # scale height to 250
            # 250 = h/ (2^n)
            # scale_n = ln(h)/ln(2) - ln(250)/ln(2)
            scale_n = math.log(pix.height, 2) - math.log(250, 2)
        else:
            scale_n = math.log(pix.width, 2) - math.log(250, 2)
        
        pix.shrink(round(scale_n))
        thumb_p = file_p.replace("PDFs", "thumbnails").replace(".pdf", "_" + str(page_no) + ".jpg")
        pix.writePNG(thumb_p)
    return(thumb_p)

In [ ]:
thumbnail_location = []
#index_file = index_file[9000:10500]
pdfs_with_errors = set()
for inx, row in index.iterrows():
    dataID_value = row["PDF Download URL"].split("/")[-1]
    pageno_value = row["PDF Page Number"]
    Data_ID.append(dataID_value)
    try:
        get_thumbnail(int(dataID_value), int(pageno_value), path)
        thumbnail_location.append("thumbnails\\"+ str(dataID_value) + "_" +str(pageno_value)+ ".jpg")
    except:
        #print(dataID_value)
        pdfs_with_errors.add(dataID_value)
        thumbnail_location.append("")
    #print("thumbnails\\"+ str(dataID_value) + "_" +str(pageno_value)+ ".jpg")

In [ ]:
print(pdfs_with_errors)
print(len(pdfs_with_errors))

In [ ]:
#index_file = index_file.head(500)
# index_file['Data ID'] = Data_ID
index['Thumbnail Location'] = thumbnail_location
#index_file['ID'] = ID
#index_file.to_csv(path + "Data_Files_v2\\book1_added.csv")

In [ ]:
col = []
for columns in index_file:
    col.append(columns)
    print(columns)

In [ ]:
index = index.sort_values(by = ['Data ID','PDF Page Number','Content Type', "Title"])
index.head()

### Add ID for all the rows 

In [ ]:
gp = index.groupby(['Data ID','PDF Page Number','Content Type'])
countID = []
for group in gp.groups:
    count = 1
    for i in gp.groups[group]:
        countID.append(count)
        count = count +1
index["countID"] = countID

In [ ]:
ID = []
for ind, row in index.iterrows():
    if "Alignment Sheet" in row["Content Type"]:
        code = "a"
    elif "Figure" in row["Content Type"]:
        code = "f"
    elif "Table" in row["Content Type"]:
        code = "t"
    else:
        print("Error with ", row["Content Type"])
    ID.append(str(row["Data ID"]) +"_"+ str(row["PDF Page Number"]) + "_" + code + str(row["countID"]))
index["ID"] = ID

In [ ]:
index.drop('countID', 
            axis=1, 
            inplace=True)

In [ ]:
index.head()

In [ ]:
index.to_csv(path + "Data_Files_v2\\index_final.csv",index=False)

In [ ]:
print(len(index))
index.shape

In [ ]:
#gp = index.groupby(['Data ID','PDF Page Number','Content Type'])
#gp = gp[gp.Title > 3]
# gp.groups

# count = 0
# for group in index.groupby(['Data ID','PDF Page Number','Content Type']):
#     #print(type(group))
#     count = count+1
# print(count)

In [ ]:
index.head()

In [ ]:
len(index)